- Antes de aplicar código solo he cambiado el nombre de las cabeceras para unificarlos
- He unido los csvs de life expectancy de Our World in data añadiendo la columna gender
- en la extracion analie algunos ficheros que descarté al estar incompletos oo no servir pa na. He convertido los csvs a dfs para las transformaciones
- Mergeo en 7BirthAttendedBySkilledPersonal los datos de la WHO(dado en Kaggle) y World Bank para ampliar. 151 valores no coinciden asi que hago su media
- He descargado 16Cardiovascular de la WHO para que fuera más completo y le he cambiado el formato para que encaje con el resto
- Otras transformaciones: Separar genero en Life expectancy y eliminar both sexes en neonatal
- En 26AirPollution separamos en varios csvs en función de la disease y en si es o no age -standarized
- En donde hay Area(36,37,38,24,25), separamos en varios csvs: Total, rural, urban
- Cambiar el nombre de las cabeceras de los csv para que queden unificados
- Explicar conjunto de datos de partida y el formato (csv) en que estaban, asignandole un número a cada uno
- He filtrado los datasets menos relevantes
- He hecho un git para importarlos a jupyter
- He tratado los datos para definir correctamente un DataFrame para cada csv separando algunos en varios DFs, eliminando columnas innecesarias, separando aquellas columnas que dan un intervalo de confianza para el valor cambiando los tipos de datos y renombrando las columnas.

In [1]:
# Miguel Roca García

**Unificación de varios CSVs a uno único**

Para añadir un nuevo CSV, solo hay que subirlo al GitHub y añadirlo en IndicatorsDataFrameNames.json
Tiene que tener las mismas cabeceras y formato que el resto.

# Importar librerías

In [2]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None #Para evitar warnings
import urllib.request
import re
import ast
pd.set_option("display.max_columns", None)#Para mostrar todas las columnas

# Importar datos

In [3]:
#Importamos de GitHub un JSON con los nombres de todos los CSV con los datos y sus futuros nombres de columna en el DataFrame
namesIndicatorJson = urllib.request.urlopen('https://raw.githubusercontent.com/MiguelRocaGarcia/DatasetTFG/master/IndicatorsDataFrameNames.json')
namesJson = namesIndicatorJson.read().decode()
indicatorsNames = ast.literal_eval(namesJson)

In [4]:
#Importamos los CSVs
dfs = {}
for name in indicatorsNames.keys():
    dfs[name] = pd.read_csv(f'https://raw.githubusercontent.com/MiguelRocaGarcia/DatasetTFG/master/{name}.csv', sep=',')

# Transformaciones

In [5]:
# Eliminamos la columna Indicator
for df in dfs.values():
    if 'Indicator' in df.columns:
        df.drop('Indicator', axis=1, inplace=True)   

In [6]:
#Separamos los valores de las columnas estimados con su intervalo de confianza
for dfName in dfs.keys():
    if '[' in str(dfs[dfName]['Value'][0]):
        dfs[dfName]['Low Value'] = dfs[dfName]['Value'].apply(lambda x: re.search('\[(.*?)-',x).group(1))
        dfs[dfName]['High Value'] = dfs[dfName]['Value'].apply(lambda x: re.search('-(.*?)\]',x).group(1))
        dfs[dfName]['Value'] = dfs[dfName]['Value'].apply(lambda x: re.search('(.*?)\[',x).group(1))

In [7]:
#Número de filas antes de la transformación
shapes = {}
for name, df in sorted(dfs.items()):
    shapes[name] = df.shape[0]

In [8]:
#Cambiar tipos de datos: de str a int o float y eliminar los NaN
for dfName in dfs.keys():
    dfs[dfName].dropna(inplace=True)  #Eliminamos los valores NaN(Necesario para que no convierta Year a float)
    dfs[dfName]['Year'] = pd.to_numeric(dfs[dfName]['Year'], errors='coerce',downcast='integer') #Convertimos Year a integer
    dfs[dfName]['Value'] = pd.to_numeric(dfs[dfName]['Value'], errors='coerce') #Convertimos value a numérico
    if ('Low Value' in dfs[dfName].columns):
        dfs[dfName]['Low Value'] = pd.to_numeric(dfs[dfName]['Low Value'], errors='coerce')
        dfs[dfName]['High Value'] = pd.to_numeric(dfs[dfName]['High Value'], errors='coerce')
    dfs[dfName].dropna(inplace=True)  #Eliminamos los valores NaN

In [9]:
# Número de filas tras la transformación
shapesAfter = {}
for name, df in sorted(dfs.items()):
    shapesAfter[name] = df.shape[0]

In [10]:
#Filas eliminadas por contener NaN
for name in sorted(shapes.keys()):
    print(name, ': ', shapes[name] - shapesAfter[name])

10_Under5MortalityRate :  0
11_IncedenceOfMalaria :  0
12_IncedenceOfTuberculosis :  0
13_HepatitisBsurfaceAntigen :  0
14_InterventionAgainstNTDs :  0
16_CardiovascularDyingProbability30_70 :  0
17_SuicideRates :  0
18_AlcoholSubstanceAbuse :  0
19_RoadTrafficDeaths :  74
1_LifeExpectancyAtBirth :  0
20_ReproductiveAgeWomen :  0
21_AdolescentBirthRate :  0
22_UniversalHealthCareCoverage :  0
24-1_Population10PercentageSDGTotal :  0
24-2_Population10PercentageSDGUrban :  0
24-3_Population10PercentageSDGRural :  0
25-1_Population25PercentageSDGTotal :  0
25-2_Population25PercentageSDGUrban :  0
25-3_Population25PercentageSDGRural :  0
26-1-1_AirPollutionDeathRateStroke :  0
26-1-2_AirPollutionDeathRateStrokeAgeStandardized :  0
26-2-1_AirPollutionDeathRateIschaemicHeartDisease :  0
26-2-2_AirPollutionDeathRateIschaemicHeartDiseaseAgeStandardized :  0
26-3-1_AirPollutionDeathRateLowerRespiratoryInfections :  0
26-3-2_AirPollutionDeathRateLowerRespiratoryInfectionsAgeStandardized :  0
26-

In [11]:
#Vemos los años disponibles para cada indicador
for name, df in sorted(dfs.items()):
  if 'Year' in df.columns:
    years = df['Year'].unique().tolist()
    print(name, '\n', years)
    print()

10_Under5MortalityRate 
 [2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950]

11_IncedenceOfMalaria 
 [2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000]

12_IncedenceOfTuberculosis 
 [2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000]

13_HepatitisBsurfaceAntigen 
 [2015]

14_InterventionAgainstNTDs 
 [2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010]

16_CardiovascularDyingProbability30_70 
 [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2

50_ConflictAndTerrorismDeaths 
 [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

51_HomicideRate 
 [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

52_GovernmentExpenditureEducation 
 [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2018, 2009, 1996, 1990, 1991, 1992, 1993, 1994, 1995, 2019]

53_GovernmentExpenditureMilitary 
 [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003]

54_GovernmentExpenditureHealth 
 [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 1880, 1890, 1900, 1910, 192

In [12]:
#Filtramos por años entre 1900 y 2019
for dfName in dfs.keys():
    dfs[dfName] = dfs[dfName][(dfs[dfName]['Year'] >= 1990)]

In [13]:
# Cambiamos los nombres de las columnas para hacerlos más significativos
for dfName in dfs.keys():
    dfs[dfName].rename(columns={'Value': indicatorsNames[dfName]}, inplace=True)
    if('Low Value' in dfs[dfName].columns):
        dfs[dfName].rename(columns={'Low Value': f'Low CI Value {indicatorsNames[dfName]}',
                                   'High Value': f'High CI Value {indicatorsNames[dfName]}'}, inplace=True)

# Unificar Conjunto de Datos

In [14]:
#Ver países que tienen valor para Life Expectancy 
countriesLE = set(dfs['1_LifeExpectancyAtBirth']['Country'].unique())

In [15]:
#Ver países que tienen Life Expectancy a NaN
countriesNotLE = set()
for df in dfs.values():
    countriesNotLE.update(df['Country'].unique())

countriesNotLE = countriesNotLE.difference(countriesLE)

In [16]:
#Cambiamos el nombre de estos países para que coincida con el nombre correcto(que tiene Life Expectancy)

changeNames = [('Bahamas, The','Bahamas'), ('Cabo Verde', 'Cape Verde'),
               ( 'Congo, Dem. Rep.','Democratic Republic of Congo'), ('Congo, Rep.', 'Congo'),
               ('Czech Republic','Czechia'), ("Côte d'Ivoire","Cote d'Ivoire"), ('Côte d’Ivoire',"Cote d'Ivoire"), 
               ("Democratic People's Republic of Korea",'North Korea'), 
               ('Democratic Republic of the Congo', 'Democratic Republic of Congo'),  ('Ethiopia PDR', 'Ethiopia'), 
               ('Federated States of Micronesia','Micronesia (country)'), ('Gambia, The', 'Gambia'), 
               ('Iran (Islamic Republic of)', 'Iran'), ('Iran, Islamic Rep.', 'Iran'), 
               ('Korea, Dem. People’s Rep.', 'North Korea'), ('Korea, Rep.', 'South Korea'), ('Lao PDR', 'Laos'),
               ("Lao People's Democratic Republic", 'Laos'), ('Micronesia (Federated States of)', 'Micronesia (country)'),
               ('Micronesia, Fed. Sts.', 'Micronesia (country)'), ('Republic of Korea', 'South Korea'),
               ('Republic of Moldova', 'Moldova'), ('Russian Federation', 'Russia'), ('Slovak Republic', 'Slovakia'),
               ('State of Palestine', 'Palestine'), ('Syrian Arab Republic', 'Syria'), ('The Bahamas', 'Bahamas'), 
               ('The Gambia', 'Gambia'), ('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom'),
               ('United Republic of Tanzania', 'Tanzania'), ('United States of America', 'United States'),
               ('Venezuela (Bolivarian Republic of)', 'Venezuela'), ('Venezuela, RB', 'Venezuela'), ('Viet Nam', 'Vietnam'),
               ('Yemen, Rep.', 'Yemen')]

for dfName in dfs.keys():
    for oldName, newName in changeNames:
        dfs[dfName].loc[dfs[dfName]['Country'] == oldName, 'Country'] = newName

In [17]:
# Separamos en dos listas con los nombres de los DFs que tienen y no tienen Gender para después hacer los merge
dfsWithGender = []
dfsWithoutGender = []
for dfName, df in dfs.items():
    if('Gender' in df.columns):
        dfsWithGender.append(dfName)
    else:
        dfsWithoutGender.append(dfName)
print('With gender: ', dfsWithGender, '\n\nWithout Gender: ', dfsWithoutGender)

With gender:  ['1_LifeExpectancyAtBirth', '8_InfantMortalityRate', '10_Under5MortalityRate', '16_CardiovascularDyingProbability30_70', '17_SuicideRates', '18_AlcoholSubstanceAbuse', '26-1-1_AirPollutionDeathRateStroke', '26-1-2_AirPollutionDeathRateStrokeAgeStandardized', '26-2-1_AirPollutionDeathRateIschaemicHeartDisease', '26-2-2_AirPollutionDeathRateIschaemicHeartDiseaseAgeStandardized', '26-3-1_AirPollutionDeathRateLowerRespiratoryInfections', '26-3-2_AirPollutionDeathRateLowerRespiratoryInfectionsAgeStandardized', '26-4-1_AirPollutionDeathRateChronicObstructivePulmonaryDisease', '26-4-2_AirPollutionDeathRateChronicObstructivePulmonaryDiseaseAgeStandardized', '26-5-1_AirPollutionDeathRateTotal', '26-5-2_AirPollutionDeathRateTotalAgeStandardized', '26-6-1_AirPollutionDeathRateTracheaBronchusLungCancers', '26-6-2_AirPollutionDeathRateTracheaBronchusLungCancersAgeStandardized', '27_MortalityRateUnsafeWash', '28_MortalityRatePoisoning', '29_TobaccoPrevalence', '47-1_PopulationAges0-14'

In [36]:
#Mergeamos por gender a partir de Life Expectancy
df_mergedByGender = dfs['1_LifeExpectancyAtBirth']
for dfName in dfsWithGender[1:]:
    df_mergedByGender = pd.merge(df_mergedByGender, dfs[dfName], 
                                 on=['Country', 'Year', 'Gender'], how='left').sort_values(['Country', 'Year', 'Gender'])

In [37]:
# Mergeamos los que no tienen gender
df_mergedWithoutGender = dfs[dfsWithoutGender[0]]
for dfName in dfsWithoutGender[1:]:
    df_mergedWithoutGender = pd.merge(df_mergedWithoutGender, dfs[dfName], 
                                      on=['Country', 'Year'], how='outer').sort_values(['Country', 'Year'])

In [38]:
# Mergemos ambos merges
df_merged = pd.merge(df_mergedByGender, df_mergedWithoutGender, 
                                      on=['Country', 'Year'], how='left').sort_values(['Country', 'Year', 'Gender'])

# Exportar a un CSV

In [22]:
df_merged.to_csv('UnifiedDataset.csv',sep=',', index=False)